In [1]:
using KerbalMath
using KerbalRemoteLogging
using KRPC
using SpaceLib
using .Engine
using .Parachute
using .Decoupler
import KRPC.Interface.SpaceCenter as SC
import KRPC.Interface.SpaceCenter.RemoteTypes as SCR
import KRPC.Interface.SpaceCenter.Helpers as SCH


In [2]:
include("setup.jl")

function register_parts(sp::Spacecraft)
    parts = SCH.Parts(sp.ves)
    core = SCH.WithTag(parts, "core")[1] |> CommandModule.ProbeCore
    e1 = SCH.WithTag(parts, "e1")[1] |> Engine.RealEngine
    e2 = SCH.WithTag(parts, "e2")[1] |> Engine.RealEngine
    chute = SCH.WithTag(parts, "chute")[1] |> Parachute.RealChute
    dec1 = SCH.WithTag(parts, "dec1")[1] |> Decoupler.RegularDecoupler
    dec2 = SCH.WithTag(parts, "dec2")[1] |> Decoupler.RegularDecoupler
    return (core, e1, e2, chute, dec1, dec2)
end

function setup_logger(sp::Spacecraft)
    @assert pwd() |> dirname |> basename == "Karman3"
    logger = KerbalRemoteLogger(;
        port=50003,
        timestring=timestring(sp),
        console_loglevel=Base.LogLevel(-1000),
        console_exclude_group=(:ProgressLogging,:time,),
        disk_directory="./run/log",
    )
    Base.global_logger(logger)
end
;

In [4]:
sc = SpaceCenter("Karman Sample Return", "10.0.0.51")
sp = add_active_vessel!(sc)
setup_logger(sp)
con = subcontrol(sp, "subcontrol")
core, e1, e2, chute, dec1, dec2 = setup(sp, con)
;

┌ Info: Hardware control loop starting
└ @ SpaceLib /home/rhahi/.julia/dev/SpaceLib/src/spacecraft.jl:217
┌ Info: Acquired spacecraft Karman 5 for control
└ @ SpaceLib /home/rhahi/.julia/dev/SpaceLib/src/spacecenter.jl:56


In [5]:

function stage0(sp::Spacecraft)
    stage!(sp)
    delay(sp.ts, 0.6, "SRB")
end

function stage1(sp::Spacecraft)
    ignite!(sp, e1; name="E1 ignition")
    stage!(sp)
    wait_for_burnout(sp, e1;
        progress=true,
        name="Stage 1",
        margin=0,
        timeout=50,
        interrupt=event!(sp, :s1),
    )
end

function stage2(sp::Spacecraft)
    ignite!(sp, e2; name="E2 ignition")
    shutdown!(e1)
    stage!(sp)
    wait_for_burnout(sp, e2;
        progress=true,
        name="Stage 2",
        margin=0,
        timeout=50,
        interrupt=event!(sp, :s2),
    )
end

function deploy(sp::Spacecraft)
    event(sp, :s3; create=true)
    periodic_subscribe(sp.ts, 5) do clock
        ref = ReferenceFrame.BCBF(sp.ves)
        h0 = SCH.Position(sp.ves, ref) |> norm
        h_prev = h0
        for now = clock
            isset(sp, :s3) && break
            h = SCH.Position(sp.ves, ref) |> norm
            h - h_prev < 0 && break
            h_prev = h
            yield()
        end
    end
    arm!(chute)
    delay(sp.ts, 1)
    if isarmed(chute)
        @info "Parachute has been armed" _group=:module
    end
end

function detach(sp::Spacecraft)
    decouple!(dec1; top=true)
    decouple!(dec2; bottom=true)
end

;

In [6]:
main = @async begin
    try
        stage0(sp)
        stage1(sp)
        stage2(sp)
        deploy(sp)
        detach(sp)
    catch e
        @error "Error has occured, to review the error, wait on this task." _group=:system
        error(e)
    end
end
main |> errormonitor

Task (runnable) @0x00007fd37237f3a0

In [ ]:
# wait(main)
# notify(sp, :s1)
# notify(sp, :s2)
# notify(sp, :s3)